In [1]:
!pip install tensorflow
!pip uninstall -y tensorflow tensorflow-gpu
!pip install numpy scipy librosa unidecode inflect librosa transformers
!pip install deeppavlov
!pip install python-telegram-bot --upgrade
!pip install dialogflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached tensorflow-2.11.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (588.3 MB)
Found existing installation: tensorflow 2.11.0
Uninstalling tensorflow-2.11.0:
  Successfully uninstalled tensorflow-2.11.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import logging

from telegram import Update
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext

import dialogflow

In [3]:
my_token = '.............'

In [4]:
updater = Updater(token=my_token) # Токен API к Telegram
dispatcher = updater.dispatcher

In [5]:
# Обработка команд
def startCommand(bot, update):
    bot.send_message(chat_id=update.message.chat_id, text='Добрый день! Чем могу помочь?')
    
def textMessage(bot, update):
    response = 'Уже работаем над Вашим вопросом' + update.message.text # формируем текст ответа
    bot.send_message(chat_id=update.message.chat_id, text=response)

In [6]:
# Enable logging
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO
)

logger = logging.getLogger()

**RuBERT** for question answering.
Здесь мы будем использовать модель, предварительно обученную на наборе данных SberQuAD

In [9]:
!python -m deeppavlov install squad_ru_bert

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
from deeppavlov import build_model, configs
model_ru = build_model(configs.squad.squad_ru_bert, download=True)

2022-12-28 17:42:42.740 INFO in 'deeppavlov.core.data.utils'['utils'] at line 95: Downloading from http://files.deeppavlov.ai/v1/squad/squad_ru_torch_bert.tar.gz to /root/.deeppavlov/squad_ru_torch_bert.tar.gz
INFO:deeppavlov.core.data.utils:Downloading from http://files.deeppavlov.ai/v1/squad/squad_ru_torch_bert.tar.gz to /root/.deeppavlov/squad_ru_torch_bert.tar.gz
100%|██████████| 1.74G/1.74G [01:36<00:00, 18.1MB/s]
2022-12-28 17:44:19.778 INFO in 'deeppavlov.core.data.utils'['utils'] at line 276: Extracting /root/.deeppavlov/squad_ru_torch_bert.tar.gz archive into /root/.deeppavlov/models
INFO:deeppavlov.core.data.utils:Extracting /root/.deeppavlov/squad_ru_torch_bert.tar.gz archive into /root/.deeppavlov/models


Downloading:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/642 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.57M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/681M [00:00<?, ?B/s]

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from 

Следующий текст скопирован с поста на хабре о полете Crew Dragon flight.

In [11]:
text = """Первая многоразовая ступень ракеты-носителя Falcon 9 успешно отделилась через две с половиной минуты после старта и автоматически приземлилась на плавучую платформу Of Course I Still Love You у берегов Флориды. Через 12 минут после запуска космический корабль Crew Dragon вышел на расчетную орбиту и отделился от второй ступени ракеты.

Сближение корабля Crew Dragon с Международной космической станцией запланировано на 31 мая. К стыковочному адаптеру на узловом модуле «Гармония» американского сегмента МКС Crew Dragon должен причалить в ручном или, при необходимости, в автоматическом режиме. Эта процедура запланирована на 10:29 по времени Восточного побережья США (17:29 по московскому времени).

В испытательном полете DM2 астронавт Херли является командиром космического корабля (spacecraft commander), а его напарник Бенкен — командир по операциям стыковки и расстыковки (joint operations commander). Фактически это означает, что именно Херли управляет Crew Dragon в полете к МКС, к которой они должны пристыковаться в течение суток после старта. Херли и Бенкен также будут выполнять необходимые для сертификации НАСА проверки систем корабля в полете.

Во время полета Херли и Бенкен провели небольшую экскурсию по Crew Dragon."""

# Source: https://habr.com/ru/news/t/504642/

In [12]:
print(text)

Первая многоразовая ступень ракеты-носителя Falcon 9 успешно отделилась через две с половиной минуты после старта и автоматически приземлилась на плавучую платформу Of Course I Still Love You у берегов Флориды. Через 12 минут после запуска космический корабль Crew Dragon вышел на расчетную орбиту и отделился от второй ступени ракеты.

Сближение корабля Crew Dragon с Международной космической станцией запланировано на 31 мая. К стыковочному адаптеру на узловом модуле «Гармония» американского сегмента МКС Crew Dragon должен причалить в ручном или, при необходимости, в автоматическом режиме. Эта процедура запланирована на 10:29 по времени Восточного побережья США (17:29 по московскому времени).

В испытательном полете DM2 астронавт Херли является командиром космического корабля (spacecraft commander), а его напарник Бенкен — командир по операциям стыковки и расстыковки (joint operations commander). Фактически это означает, что именно Херли управляет Crew Dragon в полете к МКС, к которой о

In [13]:
model_ru([text], ['Когда отделилась первая ступень?'])

[['через две с половиной минуты после старта'], [72], [1.0]]

In [14]:
model_ru([text], ['На какую дату запланирована стыковка?'])

[['на 31 мая'], [418], [0.9999997615814209]]

In [15]:
model_ru([text], ['Кто участвует в полете?'])

[['Херли'], [739], [0.9999939799308777]]

In [16]:
model_ru([text], ['Кто участвует в полете кроме астронавта Херли?'])

[['его напарник Бенкен'], [812], [0.999999463558197]]

In [17]:
reply_text = model_ru([text], ['Кто участвует в полете кроме астронавта Херли?'])
reply_text[0][0].encode('utf8')

b'\xd0\xb5\xd0\xb3\xd0\xbe \xd0\xbd\xd0\xb0\xd0\xbf\xd0\xb0\xd1\x80\xd0\xbd\xd0\xb8\xd0\xba \xd0\x91\xd0\xb5\xd0\xbd\xd0\xba\xd0\xb5\xd0\xbd'

Telegram + RuBERT for question answering.

In [18]:
# Define a few command handlers. These usually take the two arguments update and
# context. Error handlers also receive the raised TelegramError object in error.
def start(update: Update, context: CallbackContext):
    update.message.reply_text('Hi!')

def echo(update: Update, context: CallbackContext):
    txt = update.message.text

    update.message.reply_text('Ваше сообщение! ' + update.message.text)
    reply_text = model_ru([text], [update.message.text])

    #.encode('utf8')
    reply_text = reply_text[0][0] 
    
    update.message.reply_text(reply_text)

In [19]:
updater = Updater(my_token, use_context=True)
dispatcher = updater.dispatcher

# on different commands - answer in Telegram
dispatcher.add_handler(CommandHandler("start", start))
dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, echo))

# Start the Bot
updater.start_polling()
updater.idle()